# Evolutionary methods with `multiprocessing`

*At least let me have a little fun!*

Context on evolutionary algorithms from the Four Competences (on whiteboard)
- [more context here](https://towardsdatascience.com/daniel-c-dennetts-four-competences-779648bdbabc?source=friends_link&sk=15fe38a0971a25c0ddb028aec05109a4)
- **neuroevolution** = using evolutionary methods to find the parameters (or even architecture) of a neural network

Computational evolutionary methods
- general, black box optimization
- gradient free 
- work in challenging cost functions (non-linear, discontinuous etc)
- can be parallelized

Sample inefficient
- because they learn from a weak learning signal (fitness / total episode reward)
- don't learn from state / reward transitions that occur during an episode

If you want a tool for your toolbet, CMA-ES (https://en.wikipedia.org/wiki/CMA-ES) is a good choice
- adapts covariance matrix
- good up until 1,000 - 10,000 parameters
- used in World Models

## Generate, test & select

Evolutionary improvement occurs through a **generate, test & select loop**
- substrate independent

Our algorithm will:
- generate a population of parameters (neural network weights and biases)
- test these parameters in the `mountaincar` environment
- select the best performing set of parameters to use in the next generate step

Let's first setup the code for the forward pass of a neural net.  We aren't going to do any backprop, so we can do it all in `numpy`:

In [1]:
!pip install gym pygame -q

import gym
from evolution import make_env, initialize_parameters, episode

#  you can use either 'CartPole-v0' or 'MountainCarContinuous-v0'
env_id = 'CartPole-v0'
env_id = 'MountainCarContinuous-v0'
h_size = 10
env, forward, i_size, o_size = make_env(env_id)
params = initialize_parameters(i_size, h_size, o_size)

You should consider upgrading via the '/Users/adam/.pyenv/versions/3.8.5/envs/general/bin/python3.8 -m pip install --upgrade pip' command.


We end up with a dictionary of parameters with random weights:

In [2]:
params.keys()

/Users/adam/.pyenv/versions/3.8.5/envs/general/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dict_keys(['w0', 'b0', 'w1', 'b1'])

We can use the function forward to select an action using these parameters & a randomly sampled observation:

In [2]:
action = forward(env.observation_space.sample(), params)
action

/Users/adam/.pyenv/versions/3.8.5/envs/general/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[0.46199009]])

Below the machinery for saving and loading parameters is given - this is so you can run `python render_mountaincar.py` when you agent is learning:

In [3]:
from evolution import save_params, load_params

params = initialize_parameters(i_size, h_size, o_size)
save_params(params, env_id, agent_id=1)
params = load_params(env_id, agent_id=1)

saving to agents/MountainCarContinuous-v0/agent_1
loading from agents/MountainCarContinuous-v0/agent_1


Now you should be able to run (in a shell - will break your notebook kernel):

```bash
$ python render.py --env_id CartPole-v0 --agent_id 1

$ python render.py --env_id MountainCarContinuous-v0 --agent_id 1
```

![](assets/car.png)

## The components of an evolutionary algorithm

Above we have outlined the code we need to do the test step of generate/test/select.  Now we will outline the code for the generate & select steps.

For the first generation, the loop is to **generate** a population with random weights:

In [4]:
pop_size = 32
pop = [initialize_parameters(i_size, h_size, o_size) for _ in range(pop_size)]

**Test** the population in the environment:

In [6]:
from functools import partial

results = list(map(partial(episode, env_id=env_id), pop))

print(np.mean(results))
assert len(results) == pop_size

-135.66124760278075


**Select** the best performing parameters:

In [7]:
best = pop[np.argmax(results)]

We are now on the second generation - instead of sampling parameters randomly using `initialize_parameters` we use selected best from the last generation.

We still sample a new generation randomly, but now we use the results of the first generation to create the distribution to sample from.

Below we create a new generation, using the best performing member to estimate the mean.  We use an identity covariance matrix.

In [8]:
np.eye(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [9]:
num_w = 4
[np.random.normal(0, 1) for _ in range(num_w)]

np.random.multivariate_normal([0, 0, 0, 0], np.eye(4))

array([-2.34898098, -0.30588893, -0.75487664,  0.70972261])

In [10]:
def sample_params(best):
    p = {}
    for k, v in best.items():
        #  sample new parameters from a flat multivariate normal
        flat = v.flatten()
        new = np.random.multivariate_normal(flat, np.eye(flat.shape[0]), size=1)
        #  reshape the weight or bias
        p[k] = new.reshape(v.shape)
    return p

pop = [sample_params(best) for _ in range(pop_size)]

In [11]:
results = list(map(partial(episode, env_id=env_id), pop))
print(np.mean(results))

-1459.082061596898


## Practical

Take the components above and put them together:
- implement an evolutionary method using `map` (single core)
- implement an evolutionary method using `pool.map` (multi-core)

```
initialize a population of parameters

for generation in generation
    test parameters in the environment
    select the best
    generate new parameters using the best
```

In [13]:
generations = 64
pop_size = 32

#  cartpole broken
env_id = 'CartPole-v0'
env_id = 'MountainCarContinuous-v0'
pop = [initialize_parameters(i_size, h_size, o_size) for _ in range(pop_size)]
for generation in range(generations):
    #  test
    results = list(map(partial(episode, env_id=env_id), pop))
    print(np.mean(results))
    #  select
    best = pop[np.argmax(results)]
    pop = [sample_params(best) for _ in range(pop_size)]
    save_params(best, env_id, agent_id=generation)

-96.05617855834738
saving to agents/MountainCarContinuous-v0/agent_0
-1286.555787827189
saving to agents/MountainCarContinuous-v0/agent_1


/Users/adam/.pyenv/versions/3.8.5/envs/general/lib/python3.8/site-packages/gym/envs/classic_control/continuous_mountain_car.py:157: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  self.state = np.array([position, velocity], dtype=np.float32)


-3013.746492249873
saving to agents/MountainCarContinuous-v0/agent_2
-1910.8434185054357
saving to agents/MountainCarContinuous-v0/agent_3
-4561.310058945429
saving to agents/MountainCarContinuous-v0/agent_4
-5808.4653146792425
saving to agents/MountainCarContinuous-v0/agent_5
-1677.3546036664297
saving to agents/MountainCarContinuous-v0/agent_6
-2585.223913714975
saving to agents/MountainCarContinuous-v0/agent_7
-1758.8380225772728
saving to agents/MountainCarContinuous-v0/agent_8
-748.6910438321156
saving to agents/MountainCarContinuous-v0/agent_9
-2115.8641869362928
saving to agents/MountainCarContinuous-v0/agent_10
-2836.3429809222644
saving to agents/MountainCarContinuous-v0/agent_11
-4314.142912406582
saving to agents/MountainCarContinuous-v0/agent_12
-4601.559110345568
saving to agents/MountainCarContinuous-v0/agent_13
-4240.4373313317155
saving to agents/MountainCarContinuous-v0/agent_14
-5293.962920051354
saving to agents/MountainCarContinuous-v0/agent_15
-10417.819279246614
s